# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [226]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


In [227]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [228]:
# frequency table for prestige and whether or not someone was admitted
df_tab = pd.crosstab(index = df['prestige'], columns = ['admit'])
df_tab

col_0,admit
prestige,
1.0,61
2.0,148
3.0,121
4.0,67


Doesn't allow us to distinquish between admit = 0 or admit = 1

## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [229]:
dummy_ranks = pd.get_dummies(df["prestige"], prefix = "prestige")
dummy_ranks

,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0
5,0.0,1.0,0.0,0.0
6,1.0,0.0,0.0,0.0
7,0.0,1.0,0.0,0.0
8,0.0,0.0,1.0,0.0
9,0.0,1.0,0.0,0.0


#### 2.2 When modeling our class variables, how many do we need? 



Answer: Four - one for each of the 'ranks' of prestige.

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [230]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1.0':])
print handCalc.head()

   admit    gre   gpa  prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61           0.0           0.0           1.0           0.0
1      1  660.0  3.67           0.0           0.0           1.0           0.0
2      1  800.0  4.00           1.0           0.0           0.0           0.0
3      1  640.0  3.19           0.0           0.0           0.0           1.0
4      0  520.0  2.93           0.0           0.0           0.0           1.0


In [231]:
#crosstab prestige 1 admission 
prestige_1_adm = pd.crosstab(handCalc['admit'], handCalc['prestige_1.0'], rownames=['admit'])
prestige_1_adm

prestige_1.0,0.0,1.0
admit,,
0,243,28
1,93,33


In [232]:
# frequency table cutting prestige and whether or not someone was admitted
prestige_all_adm = pd.crosstab(df['admit'], df['prestige'], rownames=['admit'])
prestige_all_adm

prestige,1.0,2.0,3.0,4.0
admit,,,,
0,28,95,93,55
1,33,53,28,12


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [233]:
prestige_1_odds = 33.0/28.0
prestige_1_odds

1.1785714285714286

Of all 61 applicants from the prestige 1 colleges, 33 were admitted.  This results in admission odds of approximately 1.17 : 1.00.

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [234]:
prestige_not_1_odds = 93.0/243.0
print prestige_not_1_odds

0.382716049383


Of the 336 applicants who did not attend prestige 1 schools, 93 were admitted.  While at first glance the bigger number of applicants might suggest that non-prestige 1 schools make up a larger portion of successful applicants, the important number here is the ratio.  In this case, the success rate for non-prestige 1 school applicants is far lower, just 0.38 : 1.00.

#### 3.3 Calculate the odds ratio

#### 3.4 Write this finding in a sentenance: 

Answer: While not entirely unexpected, the difference in prestige 1 applicants and the remaining applicants is very stark.  Applicants from prestige 1 colleges enjoy a very big advantage - the ratio of 1.17 : 1.00 says, put simply, that more applicants from prestige 1 colleges were admitted than were not.  

#### 3.5 Print the cross tab for prestige_4

In [235]:
prestige_4_adm = pd.crosstab(index = handCalc['admit'], columns = handCalc['prestige_4.0'], rownames = ['admit'])
prestige_4_adm

prestige_4.0,0.0,1.0
admit,,
0,216,55
1,114,12


#### 3.6 Calculate the OR 

In [236]:
prestige_4_odds = 12.0/55.0
prestige_4_odds

0.21818181818181817

#### 3.7 Write this finding in a sentence

The ratio of applicants from prestige 4 colleges who were admitted versus those rejected is about 0.21 : 1.00.  This is far lower than the general 'non prestige 1' ratio of succesful admission of 0.38 to 1.00, but nowhere near the level prestige 1 applicants enjoy of 1.17 to 1.00.  In short, the further the prestige rank decreases from 1 to 4, the lower the odds of admission.

## Part 4. Analysis

In [237]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])
print data.head()

   admit    gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61           0.0           1.0           0.0
1      1  660.0  3.67           0.0           1.0           0.0
2      1  800.0  4.00           0.0           0.0           0.0
3      1  640.0  3.19           0.0           0.0           1.0
4      0  520.0  2.93           0.0           0.0           1.0


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [238]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [245]:
train_cols = data.columns[1:]

#### 4.2 Fit the model

In [246]:
result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 8


#### 4.3 Print the summary results

In [247]:
print result.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Mon, 12 Dec 2016   Pseudo R-squ.:                 0.08166
Time:                        19:55:10   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043      7.44e-05     0.004
gpa              0.7793      0.333      2.344      0.019         0.128     1.431
prestige_1.0    -0.0609        nan        na

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [208]:
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
print np.exp(conf)

                  2.5%     97.5%        OR
gre           1.000074  1.004372  1.002221
gpa           1.136120  4.183113  2.180027
prestige_1.0       NaN       NaN  0.940893
prestige_2.0       NaN       NaN  0.476607
prestige_3.0       NaN       NaN  0.246695
prestige_4.0       NaN       NaN  0.199023
intercept          NaN       NaN  0.022017


#### 4.5 Interpret the OR of Prestige_2

Answer: There are a few ways of looking at this.  Alone, the number suggests that prestige rank 2 has a 47% positive impact on the likelihood of admittance.  What is also suggests, in comparrison to the other ranks, is that going from prestige rank 1 to rank 2 has a 50% decrease in likelihood for admittance.  The dropoff is very steep as rank decreases, most sharply from rank 1 to rank 2, but almost as steep from rank 2 to rank 3.

#### 4.6 Interpret the OR of GPA

Answer: This is a little bit different from the prestige ranks, because the prestige ranks are dummy variables, and not continuous values.  GPA is limited but it isn't a simple binary "are you a rank 1 applicant" measurement.  For every point that the GPA increases, there is a dramatic increase for admission likelihood.  This isn't particularly surprising because a GPA score of 2.0 versus 4.0 is a vast difference in academic performance.  

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [248]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [374]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
combos.head()

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


,gre,gpa,prestige,intercept
0,220.0,2.260000,1.0,1.0
1,220.0,2.260000,2.0,1.0
2,220.0,2.260000,3.0,1.0
3,220.0,2.260000,4.0,1.0
4,220.0,2.453333,1.0,1.0


#### 5.1 Recreate the dummy variables

In [375]:
# recreate the dummy variables
dummy_values = pd.get_dummies(df['prestige'], prefix = 'prestige')
dummy_ranks.columns = ['prestige_1', 'prestige_2', 'prestige_3', 'prestige_4']
columns_data = ['gre', 'gpa','prestige', 'intercept']
# keep only what we need for making predictions

In [377]:
combos = combos[columns_data].join(dummy_ranks.ix[:, 'prestige_2':])
combos.head()

,gre,gpa,prestige,intercept,prestige_2,prestige_3,prestige_4
0,220.0,2.260000,1.0,1.0,0.0,1.0,0.0
1,220.0,2.260000,2.0,1.0,0.0,1.0,0.0
2,220.0,2.260000,3.0,1.0,0.0,0.0,0.0
3,220.0,2.260000,4.0,1.0,0.0,0.0,1.0
4,220.0,2.453333,1.0,1.0,0.0,0.0,1.0


#### 5.2 Make predictions on the enumerated dataset

In [381]:
combos_all = combos.columns[0:]

In [383]:
combos['admit_predict'] = result.predict(combos[combos_all])
combos.head()

,gre,gpa,prestige,intercept,prestige_2,prestige_3,prestige_4,admit_predict
0,220.0,2.260000,1.0,1.0,0.0,1.0,0.0,0.458354
1,220.0,2.260000,2.0,1.0,0.0,1.0,0.0,0.443271
2,220.0,2.260000,3.0,1.0,0.0,0.0,0.0,0.790097
3,220.0,2.260000,4.0,1.0,0.0,0.0,1.0,0.072336
4,220.0,2.453333,1.0,1.0,0.0,0.0,1.0,0.098155


In [384]:
combos.tail()

,gre,gpa,prestige,intercept,prestige_2,prestige_3,prestige_4,admit_predict
395,800.0,3.806667,4.0,1.0,1.0,0.0,0.0,0.913492
396,800.0,4.000000,1.0,1.0,0.0,1.0,0.0,0.922424
397,800.0,4.000000,2.0,1.0,1.0,0.0,0.0,0.932740
398,800.0,4.000000,3.0,1.0,1.0,0.0,0.0,0.928815
399,800.0,4.000000,4.0,1.0,0.0,1.0,0.0,0.908293


#### 5.3 Interpret findings for the last 4 observations

Answer: The last four observations are by themselves extremely competitive applicants - all four have perfect GRE scores and 4.0 GPAs.  This allows us to get a good comparison of what impact the prestige ranks are having - two are from prestige rank 3 colleges and two are from prestige rank 2 colleges.  The two from prestige rank 2 colleges, not surprisingly, have higher odds of admission than their cohorts from rank 3 colleges.  

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.